This Python script is used to clean 4 datasets in the raw_data folder. Including
<ol>
<li>games.csv </li>
<li>vgchartz-7_7_2020.csv</li>
<li>indie-games-developers.csv</li>
<li>video-games-developers.csv</li>
</ol>

<strong>Cleaning Processes</strong>

games.csv
1. If not present the developer place it equal to the publishers
2. drop useless columns

vgchartz-7_7_2020.csv
1. Only keep the game titles existing in games.csv, and delete the games not in games.csv. 
2. Eliminate the games without mentioning platform. 
3. Drop the useless columns.

indie-games-developers.csv
1. drop useless columns
2. collect the game names in games.csv for each developer. 3.1 (not consider series) match the game names with the substrings of "Notable games, series or franchises". Put the matched game names into list.
3.2 (consider series) if "series" exists in "Notable games, series or franchises", then take all the game names of the series into list. 

video-games-developers.csv
1. drop useless columns
2. collect the game names in games.csv for each developer. 3.1 (not consider series) match the game names with the substrings of "Notable games". Put the matched game names into list.
3.2 (consider series) if "series" exists in "Notable games", then take all the game names of the series into list. 



In [148]:
import pandas as pd
import os
from pathlib import Path

absPath = str(Path(os.path.abspath(os.getcwd())).absolute())
datasetsPath = os.path.join(absPath, "raw_datasets")
cleanedDataPath = os.path.join(absPath, "cleaned_datasets")
videogamesdevelopersUrl =  datasetsPath + '/video-games-developers.csv'



In [149]:
# parameters and URLs
gamesPath  = datasetsPath + "/games.csv"
vgchartzPath = datasetsPath + "/vgchartz-7_7_2020.csv"
indiegamesdevelopersPath = datasetsPath + "/indie-games-developers.csv"
videogamesdeveloperPath = datasetsPath + "/video-games-developers.csv"

In [150]:
games = pd.read_csv(gamesPath)
vgchartz = pd.read_csv(vgchartzPath)
indiegamesdevelopers = pd.read_csv(indiegamesdevelopersPath)
videogamesdevelopers = pd.read_csv(videogamesdeveloperPath)

games.csv cleaning

In [151]:
# If not present the developer place it equal to the publishers in games.cvs
def isNaN(string):
    return string != string

for index, row in games.iterrows():
    if isNaN(row["developers"]) :
        games.at[index,"developers"]= row["publishers"]
        
games.drop(columns=['release_na', 'release_eu', 'release_jp'])
games['platforms']

0                                      PC
1                      Xbox 360, Xbox One
2        PC, PlayStation 2, PlayStation 4
3        PC, PlayStation 2, PlayStation 4
4        PC, PlayStation 2, PlayStation 4
                       ...               
35917                                 NaN
35918                                 NaN
35919                                 NaN
35920                                 NaN
35921                                 NaN
Name: platforms, Length: 35922, dtype: object

In [152]:
gamesfilename = cleanedDataPath +'/games_cleaned.csv'
games.to_csv(gamesfilename) 

vgchartz-7_7_2020.csv Cleaning

In [153]:
# match the title with games.csv's title
vgchartz = pd.merge(vgchartz, games['title'], how ='inner', on =['title'])
#clean Series
vgchartz = vgchartz.drop(vgchartz[vgchartz.console =="Series"].index)
#drop unwanted columns
vgchartz.drop(columns=['Unnamed: 0','img', 'genre', 'publisher', 'developer', 'pal_sales', 'release_date', 'last_update'])


,title,console,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,other_sales
1,Devil May Cry,PS2,NaN,9.1,9.3,2.16,NaN,NaN,NaN,NaN
2,Devil May Cry,NS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Grand Theft Auto V,PS3,NaN,9.4,NaN,NaN,20.32,6.37,0.99,3.12
4,Grand Theft Auto V,PS4,NaN,9.7,NaN,NaN,19.39,6.06,0.60,3.02
5,Grand Theft Auto V,X360,NaN,NaN,NaN,NaN,15.86,9.06,0.06,1.42
...,...,...,...,...,...,...,...,...,...,...
26010,The 25th Ward: The Silver Case,PS4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26011,The 25th Ward: The Silver Case,PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26012,The Mind's Eclipse,PC,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26013,Without Within,PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
vgchartzfilename = cleanedDataPath +'/vgchartz_cleaned.csv'
vgchartz.to_csv(vgchartzfilename) 

**indie-games-developers.csv cleaning**

In [155]:
# match the developer with games.csv's developer
collect = games.groupby('developers')['title'].apply(list).reset_index(name='game_titles')
collect['developers'] = collect['developers'].apply(lambda x:x.lower())
indiegamesdevelopers['developers'] = indiegamesdevelopers['Developer'].apply(lambda x:x.lower())

indiegamesdevelopers = indiegamesdevelopers.merge(collect, how ='inner', on =['developers'])
indiegamesdevelopers = indiegamesdevelopers.drop(columns=['City', 'Autonomous area', 'Notes'])
indiegamesdevelopers = indiegamesdevelopers.dropna()

def match_title(ind, game_titles, Notable_games):
  matched_titles = []
  for w in game_titles:    
    if w in Notable_games:
      matched_titles.append(w)
  indiegamesdevelopers['selected_titles'][ind] = matched_titles

def match_title_include_series(ind, game_titles, Notable_games):
  matched_titles = []

  for w in game_titles:
    if 'series' in Notable_games :
      w_words=w.split()
      if len(w_words)==1:
        if w in Notable_games:
            matched_titles.append(w)
      else:
        matched_titles_largerthan2=[]
        for item in w_words:
          if item in Notable_games and item not in ['The', 'of', 'the']:
            matched_titles_largerthan2.append(item)
        if len(matched_titles_largerthan2) >1:
          matched_titles.append(w)

    elif w in Notable_games:
      matched_titles.append(w)
      
  matched_titles = list(set(matched_titles)) 
  indiegamesdevelopers['selected_titles_series'][ind] = matched_titles


indiegamesdevelopers['selected_titles']=''
indiegamesdevelopers['selected_titles_series']=''
for ind in indiegamesdevelopers.index:
  match_title(ind, indiegamesdevelopers['game_titles'][ind], indiegamesdevelopers['Notable games'][ind])
  match_title_include_series(ind, indiegamesdevelopers['game_titles'][ind], indiegamesdevelopers['Notable games'][ind])

indiegamesdevelopers = indiegamesdevelopers.drop(columns=['game_titles'])

indiegamesdevelopers

,Developer,Country,Notable games,developers,selected_titles,selected_titles_series
0,11 bit studios,Poland,AnomalyThis War of MineFrostpunk,11 bit studios,[],[]
1,11 bit studios,Poland,AnomalyThis War of MineFrostpunk,11 bit studios,"[This War of Mine, Frostpunk]","[Frostpunk, This War of Mine]"
2,ACE Team,Chile,Zeno ClashRock of Ages,ace team,"[Rock of Ages, Zeno Clash]","[Rock of Ages, Zeno Clash]"
3,Alientrap,Canada,CapsizedApotheon,alientrap,[Apotheon],[Apotheon]
4,Ambrosia Software,United States,Escape Velocity series,ambrosia software,[Escape Velocity],"[Escape Velocity, Escape Velocity Override]"
...,...,...,...,...,...,...
154,Zoetrope Interactive,Turkey,Darkness WithinDarkness Within: The Dark Lineage,zoetrope interactive,[],[]
155,Zoink,Sweden,PlayStation All-Stars IslandThe Kore Gang,zoink,[],[]
156,Zoonami,United Kingdom,ZendokuBonsai Barber,zoonami,"[Bonsai Barber, Zendoku]","[Zendoku, Bonsai Barber]"
157,ZootFly,Slovenia,First Battalion,zootfly,[],[]


In [156]:
indiegamesdevelopersfilename = cleanedDataPath +'/indiegamesdevelopers_cleaned.csv'
indiegamesdevelopers.to_csv(indiegamesdevelopersfilename) 

**video-games-developers.csv cleaning**

In [157]:
videogamesdevelopers.rename(columns={'Notable games, series or franchises': 'Notable games'}, inplace=True)


# match the developer with games.csv's developer
collect = games.groupby('developers')['title'].apply(list).reset_index(name='game_titles')
collect.rename(columns={'developers': 'Developer'}, inplace=True)

collect['Developer'] = collect['Developer'].apply(lambda x:x.lower())
videogamesdevelopers['Developer'] = videogamesdevelopers['Developer'].apply(lambda x:x.lower())


developers = videogamesdevelopers.merge(collect, how ='inner', on =['Developer'])
developers = developers.drop(columns=['City', 'Administrative division', 'Country','Est.','Notes'])
developers = developers.dropna()

def match_title(ind, game_titles, Notable_games):
  matched_titles = []
  for w in game_titles:    
    if w in Notable_games:
      matched_titles.append(w)
  developers['selected_titles'][ind] = matched_titles

def match_title_include_series(ind, game_titles, Notable_games):
  matched_titles = []
  for w in game_titles:
    if 'series' in Notable_games :
      w_words=w.split()
      for item in w_words:
        if item in Notable_games:
          matched_titles.append(w)

    elif w in Notable_games:
      matched_titles.append(w)
      
  matched_titles = list(set(matched_titles)) 
  developers['selected_titles_series'][ind] = matched_titles

developers['selected_titles']=''
developers['selected_titles_series']=''
for ind in developers.index:
  match_title(ind, developers['game_titles'][ind], developers['Notable games'][ind])
  match_title_include_series(ind, developers['game_titles'][ind], developers['Notable games'][ind])


  
developers = developers.drop(columns=['game_titles'])
developers

,Developer,Notable games,selected_titles,selected_titles_series
0,0verflow,School DaysSummer DaysCross Days,[],[]
1,11 bit studios,Frostpunk,[],[]
2,11 bit studios,Frostpunk,[Frostpunk],[Frostpunk]
3,1c company,King's Bounty: Warriors of the North,[],[]
4,2k czech,MafiaMafia II,[Mafia II],[Mafia II]
...,...,...,...,...
627,zipper interactive,SOCOM U.S. Navy SEALs seriesMechWarrior 3,[MechWarrior 3],"[SOCOM: U.S. Navy SEALs, SOCOM 3: U.S. Navy SE..."
628,zombie studios,Spec Ops: Rangers Lead the WayBlacklight: Retr...,[Blacklight: Retribution],[Blacklight: Retribution]
629,zootfly,Marlow Briggs and the Mask of Death,[],[]
630,zootfly,Marlow Briggs and the Mask of Death,[],[]


In [158]:
videogamesdevelopersfilename = cleanedDataPath +'/videogamesdevelopers_cleaned.csv'
developers.to_csv(videogamesdevelopersfilename) 